#### COLAB에서 실행(pytorch)
TabNet은 Tabular Data(표 형식 데이터)를 처리하기 위해 설계된 딥러닝 모델입니다. TabNet은 특히 표 형식의 데이터(예: 엑셀 시트, 데이터베이스 테이블 등)에서 좋은 성능을 발휘합니다.

In [ ]:
!pip install pytorch-tabnet

In [ ]:
from google.colab import files
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from pytorch_tabnet.tab_model import TabNetRegressor
import torch
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [ ]:
uploaded = files.upload()

Saving insurance.csv to insurance (1).csv


In [ ]:
df = pd.read_csv('insurance.csv')
X = df.drop(columns=['charges'])
y = df['charges']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# 변수가 다른 변수들과 스케일이 달라서 영향을 많이 받아 학습이 제대로 이루어지지 않을 수 있어서 정규화 수행.
# 이상치에 민감하지 않도록 MinMaxScaler를 사용하여 정규화.

# Pipeline 정의
# 수치형 변수의 전처리
numeric_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler())
])

# 범주형 변수의 전처리
categorical_transformer = Pipeline(steps=[

    # 만약 학습할 때는 있지만 테스트 데이터에서 새로운 범주가 발생할 경우
    # handle_unknown='ignore' 옵션을 통해 이를 무시하고 변환 작업을 진행
    # 이 옵션을 사용함으로써 모델이 학습할 때와 테스트할 때의 일관성을 유지하면서 처리할 수 있음.
    # (이름, 변환기)
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# 전체 전처리 파이프라인 정의
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, ['age', 'bmi', 'children']),  # 수치형 변수
        ('cat', categorical_transformer, ['sex', 'smoker', 'region'])  # 범주형 변수
    ])

# 최종 Pipeline 정의 (전처리 + 모델)
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Train 데이터에 대해서만 fit을 적용
pipeline.fit(X_train)

# Train 데이터와 Test 데이터 각각에 대해 transform 수행
X_train = pipeline.transform(X_train)
X_test = pipeline.transform(X_test)

# 컬럼명 재구성
# 수치형 변수와 범주형 변수의 컬럼명을 다시 가져오기
numeric_features = preprocessor.transformers_[0][2]
categorical_features = preprocessor.transformers_[1][1]['onehot'].get_feature_names_out(['sex', 'smoker', 'region'])

# 전체 특성의 컬럼명 재구성
feature_names = list(numeric_features) + list(categorical_features)

X_train = pd.DataFrame(X_train, columns=feature_names)
X_test = pd.DataFrame(X_test, columns=feature_names)

# 데이터 변환: 1D -> 2D 변환
y_train = y_train.values.reshape(-1, 1)
y_test = y_test.values.reshape(-1, 1)

In [ ]:
# TabNet 모델 정의
model = TabNetRegressor(verbose=0, seed=42, optimizer_fn=torch.optim.Adam)

# 모델 학습
model.fit(
    X_train=X_train.values,
    y_train=y_train,
    eval_set=[(X_test.values, y_test)], # 평가 세트

    # 조기 종료를 위한 patience 설정.
    # 이 값만큼의 에포크 동안 성능 개선이 없으면 학습을 조기 종료합니다.
    patience=100,
    max_epochs=1000,
    eval_metric=['rmse']
)

# 예측
y_pred = model.predict(X_test.values)

# 평가
r2 = r2_score(y_test, y_pred)
print(f'R² Score: {r2:.4f}')

# 모델 학습이 max_epochs 설정값인 1000에 도달했지만, 가장 좋은 성능을 보인 에포크는 981임.
# TabNetRegressor는 학습 도중 최적의 성능을 보인 에포크(여기서는 981)의 가중치를 자동으로 사용하여 모델을 저장.

Stop training because you reached max_epochs = 1000 with best_epoch = 981 and best_val_0_rmse = 6321.46994
R² Score: 0.7426


/usr/local/lib/python3.10/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
